### Project Info

In this project, you will use deep learning to predict forest cover type (the most common kind of tree cover) based only on cartographic variables. The actual forest cover type for a given 30 x 30 meter cell was determined from US Forest Service (USFS) Region 2 Resource Information System data. The covertypes are the following:

    Spruce/Fir
    Lodgepole Pine
    Ponderosa Pine
    Cottonwood/Willow
    Aspen
    Douglas-fir
    Krummholz

Independent variables were then derived from data obtained from the US Geological Survey and USFS. The data is raw and has not been scaled or preprocessed for you. It contains binary columns of data for qualitative independent variables such as wilderness areas and soil type.

This study area includes four wilderness areas located in the Roosevelt National Forest of northern Colorado. These areas represent forests with minimal human-caused disturbances, so existing forest cover types are mainly a result of ecological processes rather than forest management practices.
Project Objectives:

    1. Develop one or more classifiers for this multi-class classification problem.
    2. Use TensorFlow with Keras to build your classifier(s).
    3. Use your knowledge of hyperparameter tuning to improve the performance of your model(s).
    4. Test and analyze performance.
    5. Create clean and modular code.

Prerequisites:

All lessons, articles, and previous projects in Build Deep Learning Models with TensorFlow

Forum https://discuss.codecademy.com/c/project/portfolio-project-forest-cover-classification-pro/1917


In [29]:
# load modules

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
#from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# for grid search
#from keras.wrappers.scikit_learn import KerasClassifier
# DEPRECATED. Use [Sci-Keras](https://github.com/adriangb/scikeras) instead.
# See https://www.adriangb.com/scikeras/stable/migration.html
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# for random search
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

In [2]:
# load data
data = pd.read_csv("cover_data.csv")

# examine data
data.shape

(581012, 55)

In [3]:
data.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [5]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [6]:
# split into features and labels

feature_names = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology",
            "Vertical_Distance_To_Hydrology", "Hillshade_9am"]
features = data.loc[:, feature_names]

labels = data.iloc[:, -1]

features.shape

(581012, 6)

In [7]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 6 columns):
 #   Column                            Non-Null Count   Dtype
---  ------                            --------------   -----
 0   Elevation                         581012 non-null  int64
 1   Aspect                            581012 non-null  int64
 2   Slope                             581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology  581012 non-null  int64
 4   Vertical_Distance_To_Hydrology    581012 non-null  int64
 5   Hillshade_9am                     581012 non-null  int64
dtypes: int64(6)
memory usage: 26.6 MB


In [8]:
# split into training and test set

features_train, features_test, labels_train, labels_test = train_test_split(
        features, labels, train_size = 0.8, random_state = 22)

In [9]:
# standardize columns
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_train_scaled = pd.DataFrame(features_train_scaled, 
                                     columns = features_train.columns)

features_test_scaled = scaler.transform(features_test)
features_test_scaled = pd.DataFrame(features_test_scaled, 
                                     columns = features_test.columns)

In [10]:
# examine the scaled data
features_train_scaled.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Hillshade_9am
count,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05,4.648090e+05
mean,6.717593e-16,-7.035037e-17,3.134733e-16,8.548265e-17,3.836437e-16,1.925980e-16
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-3.926878e+00,-1.389191e+00,-1.880874e+00,-1.267244e+00,-3.761006e+00,-7.929285e+00
25%,-5.353621e-01,-8.711493e-01,-6.802273e-01,-7.588987e-01,-6.755309e-01,-5.288846e-01
50%,1.286610e-01,-2.548588e-01,-1.466068e-01,-2.411393e-01,-2.984172e-01,2.186306e-01
75%,7.284238e-01,9.330636e-01,5.204189e-01,5.402065e-01,3.872440e-01,7.045155e-01
max,3.209585e+00,1.826238e+00,6.923865e+00,5.308299e+00,9.506538e+00,1.564158e+00


In [11]:
# examine the scaled data
features_test_scaled.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Hillshade_9am
count,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000,116203.000000
mean,0.007223,0.005483,0.003165,0.004641,0.000843,-0.000826
std,0.997728,0.997911,0.994835,1.002235,0.996342,1.002722
min,-3.884038,-1.389191,-1.880874,-1.267244,-3.521025,-7.929285
25%,-0.528222,-0.862218,-0.680227,-0.758899,-0.675531,-0.528885
50%,0.139371,-0.254859,-0.146607,-0.241139,-0.281276,0.218631
75%,0.733779,0.941995,0.520419,0.568448,0.387244,0.704516
max,3.209585,1.826238,6.790460,5.275351,9.437972,1.564158


In [12]:
# current labels from 1 - 7, shift to 0 - 6
# using 'LabelEncoder'
#le = LabelEncoder()
#labels_train = pd.DataFrame(le.fit_transform(labels_train), columns = ["class"])
#labels_test = pd.DataFrame(le.transform(labels_test), columns = ["class"])
labels_train = labels_train - 1
labels_test = labels_test - 1

In [13]:
#####
### Building model
#####

model = Sequential()

# input layer
model.add(InputLayer(features_train_scaled.shape[1]))

# hidden layer
model.add(Dense(64, activation = "relu"))
model.add(Dense(12, activation = "relu"))

# output layer
# seven different categories of trees
model.add(Dense(7, activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                448       
                                                                 
 dense_1 (Dense)             (None, 12)                780       
                                                                 
 dense_2 (Dense)             (None, 7)                 91        
                                                                 
Total params: 1,319
Trainable params: 1,319
Non-trainable params: 0
_________________________________________________________________


2023-02-15 16:01:55.550218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# compile model
# n.b. 'categorical_crossentropy' works on one-hot encoded target, while
# 'sparse_categorical_crossentropy' works on integer target. See:
# https://stackoverflow.com/questions/61550026/valueerror-shapes-none-1-and-none-3-are-incompatible
# https://www.kaggle.com/general/197993

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam",
              metrics = ["accuracy"])

In [16]:
# fit model to training set
# early stopping rule
callback = EarlyStopping(monitor = 'loss', patience = 5)

history = model.fit(features_train_scaled, labels_train, epochs = 12,
                    batch_size = 100, verbose = 1, callbacks = [callback])

# evaluate the model
loss, acc = model.evaluate(features_test_scaled, labels_test)

Epoch 1/12
4649/4649 [==============================] - 7s 1ms/step - loss: 0.6460 - accuracy: 0.7176
Epoch 2/12
4649/4649 [==============================] - 8s 2ms/step - loss: 0.6453 - accuracy: 0.7174
Epoch 3/12
4649/4649 [==============================] - 11s 2ms/step - loss: 0.6447 - accuracy: 0.7184
Epoch 4/12
4649/4649 [==============================] - 9s 2ms/step - loss: 0.6440 - accuracy: 0.7178
Epoch 5/12
4649/4649 [==============================] - 7s 2ms/step - loss: 0.6438 - accuracy: 0.7183
Epoch 6/12
4649/4649 [==============================] - 6s 1ms/step - loss: 0.6431 - accuracy: 0.7185
Epoch 7/12
4649/4649 [==============================] - 10s 2ms/step - loss: 0.6425 - accuracy: 0.7187
Epoch 8/12
4649/4649 [==============================] - 9s 2ms/step - loss: 0.6421 - accuracy: 0.7192
Epoch 9/12
4649/4649 [==============================] - 8s 2ms/step - loss: 0.6418 - accuracy: 0.7190
Epoch 10/12
4649/4649 [==============================] - 6s 1ms/step - loss: 0.6

### Grid Search

Using 'KerasClassifier' and 'GridSearchCV'

In [23]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    # input layer
    model.add(InputLayer(features_train_scaled.shape[1]))
    # hidden layer
    model.add(Dense(64, activation = "relu"))
    model.add(Dense(12, activation = "sigmoid"))
    # output layer
    # seven different categories of trees
    model.add(Dense(7, activation = "softmax"))
    # Compile model
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam",
              metrics = ["accuracy"])
    return model

In [24]:
# fix random seed for reproducibility
seed = 22
tf.random.set_seed(seed)

In [25]:
# create model
model = KerasClassifier(model = create_model, verbose = 0)

In [26]:
# define the grid search parameters
batch_size = [60, 80, 100]
epochs = [4, 12, 20]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [27]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(features_train_scaled, labels_train, verbose = 1)

2023-02-15 16:45:21.253359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 16:45:21.253573: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 16:45:21.253761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Epoch 1/4
Epoch 1/12
Epoch 1/4
Epoch 1/4
5165/5165 [==============================] - 15s 3ms/step - loss: 0.7743 - accuracy: 0.6857   >............................] - ETA: 12s - loss: 1.1547 - accuracy: 0.557....] - ETA: 12s - loss: 
Epoch 2/4
5165/5165 [==============================] - 15s 3ms/step - loss: 0.7832 - accuracy: 0.6796
Epoch 2/12
5165/5165 [==============================] - 15s 3ms/step - loss: 0.7723 - accuracy: 0.6816
Epoch 2/4
5165/5165 [==============================] - 15s 3ms/step - loss: 0.6926 - accuracy: 0.7040
Epoch 3/4
5165/5165 [==============================] - 15s 3ms/step - loss: 0.6912 - accuracy: 0.7034
Epoch 3/4
5165/5165 [==============================] - 15s 3ms/step - loss: 0.6893 - accuracy: 0.7045
Epoch 3/12
5165/5165 [==============================] - 15s 3ms/step - loss: 0.6895 - accuracy: 0.7036
Epoch 3/4
5165/5165 [==============================] - 14s 3ms/step - loss: 0.6814 - accuracy: 0.7069=>..................] - ETA: 8s - loss: 0.6792 - a

3460/5165 [===================>..........] - ETA: 4s - loss: 0.6535 - accuracy: 0.7147

3687/5165 [====================>.........] - ETA: 4s - loss: 0.6535 - accuracy: 0.7143

5093/5165 [============================>.] - ETA: 0s - loss: 0.6908 - accuracy: 0.7040

...

3874/3874 [==============================] - 10s 2ms/step - loss: 0.6586 - accuracy: 0.7148
Epoch 10/12
3874/3874 [==============================] - 10s 3ms/step - loss: 0.6741 - accuracy: 0.7083
Epoch 5/12
5165/5165 [==============================] - 13s 3ms/step - loss: 0.6484 - accuracy: 0.7188
Epoch 19/20
3874/3874 [==============================] - 10s 2ms/step - loss: 0.6566 - accuracy: 0.7156
Epoch 11/12
3874/3874 [==============================] - 9s 2ms/step - loss: 0.6691 - accuracy: 0.7097
Epoch 6/12
5165/5165 [==============================] - 12s 2ms/step - loss: 0.6476 - accuracy: 0.7184
Epoch 20/20
3874/3874 [==============================] - 9s 2ms/step - loss: 0.6545 - accuracy: 0.7168
Epoch 12/12
3874/3874 [==============================] - 10s 3ms/step - loss: 0.6647 - accuracy: 0.7115
Epoch 7/12
3874/3874 [==============================] - 12s 3ms/step - loss: 0.6614 - accuracy: 0.7128
Epoch 8/12
3874/3874 [==============================] - 12s 3ms/step - loss: 0.81

3874/3874 [==============================] - 15s 4ms/step - loss: 0.6742 - accuracy: 0.7069
Epoch 5/20
3874/3874 [==============================] - 14s 4ms/step - loss: 0.6551 - accuracy: 0.7168/3874 [====>.........................] - ETA: 15s - loss: 0.6579 - accuracy: 0.713...] - ETA: 7s - loss: 0.6561 - accuracy: 0.716...............] - ETA: 16s - loss: 0.6588 - accuracy: 0.713s: 0.6574 - accuracy0/3874 [====>.........................] - ETA: 13s - loss: 0.6709 - accuracy: 074 [==========>...................] - ETA: 10s - loss: 0.6558 - accuracy: 0.715s - loss: 0.6559 - accuracyloss: 0.6550 - accuracy: 0./3874 [==============>...............] - ETA: 6s - loss: 0.6677 - accuracy: 0.716546 - accuracy: 0.71TA: 0s - loss: 0.6550 - accuracy: 0.71
Epoch 11/12
3874/3874 [==============================] - 13s 3ms/step - loss: 0.6563 - accuracy: 0.7143====================>......] - ETA: 2s - loss: 0.6564 - accuracy: 0.7=========================>....] - ETA: 1s - loss: 0.6567 - accuracy: 0.71

3874/3874 [==============================] - 13s 3ms/step - loss: 0.6477 - accuracy: 0.7183s - loss: 0.7058 - accuracy: 0.70==============>..] - ETA: 0s - loss: 0.6503 - accuracy: 0.71
Epoch 19/20
3874/3874 [==============================] - 12s 3ms/step - loss: 0.6502 - accuracy: 0.7176
Epoch 14/20
3874/3874 [==============================] - 11s 3ms/step - loss: 0.6462 - accuracy: 0.717674 [=======>......................] - ETA: 8s - loss: 0.6474 - accuracy: 0.71loss: 0.6480 - accuracy: 0.7==>.................] - ETA: 6s - loss: 0.6473 - accuracy: 0.71 - ETA: 5s - loss: 0.6474 - accuracy: 0.7 - accuracy: 0.
Epoch 20/20
3874/3874 [==============================] - 11s 3ms/step - loss: 0.6491 - accuracy: 0.7177
Epoch 15/20
3099/3099 [==============================] - 9s 3ms/step - loss: 0.6844 - accuracy: 0.7075
Epoch 4/4
3874/3874 [==============================] - 14s 4ms/step - loss: 0.6483 - accuracy: 0.7183
Epoch 16/20
3874/3874 [==============================] - 10s 3ms/step - lo

3099/3099 [==============================] - 7s 2ms/step - loss: 0.6637 - accuracy: 0.7117
Epoch 8/20
3099/3099 [==============================] - 7s 2ms/step - loss: 0.6594 - accuracy: 0.7140
Epoch 10/12
3099/3099 [==============================] - 8s 3ms/step - loss: 0.6614 - accuracy: 0.7126
Epoch 9/20
3099/3099 [==============================] - 11s 3ms/step - loss: 0.8066 - accuracy: 0.6738
Epoch 2/20
3099/3099 [==============================] - 8s 3ms/step - loss: 0.6577 - accuracy: 0.7144
Epoch 11/12
3099/3099 [==============================] - 10s 3ms/step - loss: 0.8214 - accuracy: 0.6696
Epoch 2/20
3099/3099 [==============================] - 11s 3ms/step - loss: 0.6594 - accuracy: 0.7131
Epoch 10/20
3099/3099 [==============================] - 11s 4ms/step - loss: 0.6968 - accuracy: 0.7021
Epoch 3/20
3099/3099 [==============================] - 12s 4ms/step - loss: 0.7030 - accuracy: 0.7020
Epoch 3/20
3099/3099 [==============================] - 11s 3ms/step - loss: 0.6574 -

5811/5811 [==============================] - 6s 1ms/step - loss: 0.6406 - accuracy: 0.7220


In [28]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.718702 using {'batch_size': 80, 'epochs': 20}
0.709352 (0.000234) with: {'batch_size': 60, 'epochs': 4}
0.715223 (0.000527) with: {'batch_size': 60, 'epochs': 12}
0.718338 (0.000346) with: {'batch_size': 60, 'epochs': 20}
0.709485 (0.000315) with: {'batch_size': 80, 'epochs': 4}
0.715700 (0.002337) with: {'batch_size': 80, 'epochs': 12}
0.718702 (0.000477) with: {'batch_size': 80, 'epochs': 20}
0.709722 (0.000726) with: {'batch_size': 100, 'epochs': 4}
0.717032 (0.000642) with: {'batch_size': 100, 'epochs': 12}
0.716666 (0.001786) with: {'batch_size': 100, 'epochs': 20}


### Random search

using 'RandomizedSearchCV'

In [31]:
param_grid = {'batch_size': randint(20, 500), 'epochs': randint(5, 30)}

# 'n_iter' gives number of parameter combinations sampled
random_grid = RandomizedSearchCV(estimator = model, param_distributions = param_grid,
                          n_jobs = -1, cv = 3, n_iter = 5)
random_grid_result = random_grid.fit(features_train_scaled, labels_train, verbose = 1)

2023-02-15 17:23:11.343928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 17:23:11.344124: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 17:23:11.349847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

Epoch 1/5
Epoch 1/17
Epoch 1/5
Epoch 1/5
1174/1174 [==============================] - 6s 3ms/step - loss: 0.9356 - accuracy: 0.6511
Epoch 2/5
1174/1174 [==============================] - 6s 3ms/step - loss: 0.9312 - accuracy: 0.6438
Epoch 2/5
1174/1174 [==============================] - 6s 3ms/step - loss: 1.0595 - accuracy: 0.5998
Epoch 2/5
1174/1174 [==============================] - 4s 3ms/step - loss: 0.7405 - accuracy: 0.6930
Epoch 3/5
1161/1161 [==============================] - 3s 3ms/step - loss: 0.7728 - accuracy: 0.6941
Epoch 3/17
1174/1174 [==============================] - 4s 3ms/step - loss: 0.7378 - accuracy: 0.6930
Epoch 3/5
1174/1174 [==============================] - 4s 3ms/step - loss: 0.7632 - accuracy: 0.6912
Epoch 3/5
1161/1161 [==============================] - 3s 3ms/step - loss: 0.7246 - accuracy: 0.6968
Epoch 4/17
1174/1174 [==============================] - 3s 3ms/step - loss: 0.7028 - accuracy: 0.6992
Epoch 4/5
1174/1174 [==============================] - 3s 

1095/1095 [==============================] - 3s 3ms/step - loss: 0.6625 - accuracy: 0.7144
Epoch 13/21
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6630 - accuracy: 0.7122
Epoch 13/17
1095/1095 [==============================] - 5s 3ms/step - loss: 1.0273 - accuracy: 0.6130
Epoch 2/21
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6590 - accuracy: 0.7143
Epoch 13/17


1095/1095 [==============================] - 3s 3ms/step - loss: 0.6607 - accuracy: 0.7148
Epoch 14/21
1095/1095 [==============================] - 3s 3ms/step - loss: 0.7680 - accuracy: 0.6904
Epoch 3/21
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6610 - accuracy: 0.7130
Epoch 14/17
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6576 - accuracy: 0.7147
Epoch 14/17
1095/1095 [==============================] - 3s 3ms/step - loss: 0.7225 - accuracy: 0.6957
Epoch 4/21
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6593 - accuracy: 0.7136
Epoch 15/17


1161/1161 [==============================] - 3s 3ms/step - loss: 0.6563 - accuracy: 0.7153
Epoch 15/17
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6578 - accuracy: 0.7143
Epoch 16/17
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6552 - accuracy: 0.7160
Epoch 16/17
1095/1095 [==============================] - 3s 3ms/step - loss: 0.6561 - accuracy: 0.7163
Epoch 17/21
1095/1095 [==============================] - 3s 3ms/step - loss: 0.6954 - accuracy: 0.7042
Epoch 6/21
1161/1161 [==============================] - 3s 3ms/step - loss: 0.6567 - accuracy: 0.7148
Epoch 17/17
1095/1095 [==============================] - 3s 3ms/step - loss: 0.6552 - accuracy: 0.7171
Epoch 18/21
1095/1095 [==============================] - 3s 3ms/step - loss: 0.6896 - accuracy: 0.7060
Epoch 7/21
1095/1095 [==============================] - 3s 2ms/step - loss: 0.6541 - accuracy: 0.7175
Epoch 19/21
1095/1095 [==============================] - 3s 2ms/step - loss: 0.6855 - a

1527/1527 [==============================] - 7s 3ms/step - loss: 0.8827 - accuracy: 0.6589
Epoch 2/21


2236/4625 [=============>................] - ETA: 6s - loss: 0.6782 - accuracy: 0.7084

1527/1527 [==============================] - 5s 3ms/step - loss: 0.7213 - accuracy: 0.6965
Epoch 3/21
4625/4625 [==============================] - 13s 3ms/step - loss: 0.6800 - accuracy: 0.7067
Epoch 4/25
1527/1527 [==============================] - 4s 3ms/step - loss: 0.6969 - accuracy: 0.7033
Epoch 4/21
4625/4625 [==============================] - 13s 3ms/step - loss: 0.6630 - accuracy: 0.7123
Epoch 7/25
4625/4625 [==============================] - 13s 3ms/step - loss: 0.6767 - accuracy: 0.7077
Epoch 5/25
1527/1527 [==============================] - 4s 3ms/step - loss: 0.6806 - accuracy: 0.7077acy: 0
Epoch 6/21
4625/4625 [==============================] - 11s 2ms/step - loss: 0.6743 - accuracy: 0.7076
Epoch 5/25
1527/1527 [==============================] - 4s 2ms/step - loss: 0.6766 - accuracy: 0.7091
Epoch 7/21
4625/4625 [==============================] - 11s 2ms/step - loss: 0.6714 - accuracy: 0.7090
Epoch 6/25
1527/1527 [==============================] - 4s 2ms/step - loss: 0.6699

4625/4625 [==============================] - 11s 2ms/step - loss: 0.6465 - accuracy: 0.7185==================>.] - ETA: 0s - loss: 0.6465 - accuracy: 0.718
Epoch 21/25
4625/4625 [==============================] - 12s 3ms/step - loss: 0.6467 - accuracy: 0.7187
Epoch 19/25
1527/1527 [==============================] - 4s 3ms/step - loss: 0.6934 - accuracy: 0.7038
Epoch 5/21
1527/1527 [==============================] - 4s 2ms/step - loss: 0.6874 - accuracy: 0.7057- loss: 0.6447 - accuracy: 0s - loss: 0.6467 - accuracy:...] - ETA: 8s - loss: 0.6474 - accuracy: 
Epoch 6/21
1527/1527 [==============================] - 4s 3ms/step - loss: 0.6829 - accuracy: 0.70716468 - accuracy: 0.71
Epoch 7/21
4625/4625 [==============================] - 11s 2ms/step - loss: 0.6480 - accuracy: 0.7175
Epoch 19/25
4625/4625 [==============================] - 11s 2ms/step - loss: 0.6458 - accuracy: 0.7190
Epoch 22/25
4625/4625 [==============================] - 11s 2ms/step - loss: 0.6458 - accuracy: 0.718625 [

6938/6938 [==============================] - 8s 1ms/step - loss: 0.6482 - accuracy: 0.7184
Epoch 12/25
6938/6938 [==============================] - 9s 1ms/step - loss: 0.6468 - accuracy: 0.7188
Epoch 13/25
6938/6938 [==============================] - 9s 1ms/step - loss: 0.6455 - accuracy: 0.7191
Epoch 14/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6446 - accuracy: 0.7195
Epoch 15/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6436 - accuracy: 0.7194
Epoch 16/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6428 - accuracy: 0.7193
Epoch 17/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6420 - accuracy: 0.7195
Epoch 18/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6414 - accuracy: 0.7203
Epoch 19/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6407 - accuracy: 0.7199
Epoch 20/25
6938/6938 [==============================] - 8s 1ms/step - loss: 0.6402 -

In [32]:
# summarize results
print("Best: %f using %s" % (random_grid_result.best_score_, random_grid_result.best_params_))
means = random_grid_result.cv_results_['mean_test_score']
stds = random_grid_result.cv_results_['std_test_score']
params = random_grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))                      

Best: 0.720135 using {'batch_size': 67, 'epochs': 25}
0.706972 (0.000486) with: {'batch_size': 264, 'epochs': 5}
0.716036 (0.000821) with: {'batch_size': 267, 'epochs': 17}
0.716378 (0.000735) with: {'batch_size': 283, 'epochs': 21}
0.720135 (0.000894) with: {'batch_size': 67, 'epochs': 25}
0.716376 (0.000830) with: {'batch_size': 203, 'epochs': 21}
